# Quant Techniques Analyses
---
### Showing most important insights about models trained via Pruning techs.

**Imports**

In [1]:
 %matplotlib
from utils.libs.all_imports_quant_analyses import *

Using matplotlib backend: Qt5Agg
No module named 'googleapiclient'


### Load Compressions Datasets
---

**Load both baseline dataset and compression dataset**

In [2]:
baseline_df, cmprss_df = load_full_cmprss_dataset()
cmprss_df.head(5)

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2,cmprss-class-3,prune_rate_intervals
0,1603478755-305517,-,229204.0,100.0,58.145780,27.979004,0.138523,0.000006,0.998769,Baseline:hf=75:hl=10,SIREN,SIREN,SIREN
1,1603478755-305517,-,229204.0,100.0,56.492704,27.979004,0.138523,0.000009,0.998202,Baseline:hf=75:hl=10,SIREN,SIREN,SIREN
2,1603478755-305517,-,229204.0,100.0,58.214385,27.979004,0.138523,0.000006,0.999085,Baseline:hf=75:hl=10,SIREN,SIREN,SIREN
3,1603478755-305517,-,252004.0,100.0,53.486111,30.762207,0.125990,0.000018,0.997542,Baseline:hf=75:hl=11,SIREN,SIREN,SIREN
4,1603478755-305517,-,252004.0,100.0,58.401365,30.762207,0.125990,0.000005,0.999003,Baseline:hf=75:hl=11,SIREN,SIREN,SIREN


**Get apart some useful sub-datasets as Jpeg and others, from just above fetched compression dataset.**

In [3]:
jpeg_df = cmprss_df[cmprss_df["cmprss-class-2"] == "Jpeg".upper()]
siren_bsln_df = cmprss_df[cmprss_df["cmprss-class-2"] == "Siren".upper()]
agp_df = cmprss_df[cmprss_df["cmprss-class-2"] == "Agp".upper()]

jpeg_df["prune_rate_intervals"] = ["JPEG"] * jpeg_df.shape[0]
siren_bsln_df["prune_rate_intervals"] = ["SIREN"] * siren_bsln_df.shape[0]

In [4]:
siren_bsln_df = pd.concat(
    [siren_bsln_df, agp_df],
    axis = 0, ignore_index = True
)

In [5]:
agp_df.head(5)

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2,cmprss-class-3,prune_rate_intervals
401,2020.12.10-044629,2020.11.29-231149,52040.0,61.967135,45.815071,6.352539,0.610108,0.000105,0.989299,AGP:38,AGP,AGP,35-40
402,2020.12.10-044629,2020.11.29-231149,52036.0,61.962372,45.284305,6.352051,0.610155,0.000119,0.988715,AGP:38,AGP,AGP,35-40
403,2020.12.09-085652,2020.11.29-231149,56952.0,67.816147,44.789306,6.952148,0.557487,0.000133,0.990357,AGP:32,AGP,AGP,30-35
404,2020.12.08-191157,2020.11.29-231149,56952.0,67.816147,45.153248,6.952148,0.557487,0.000122,0.990075,AGP:32,AGP,AGP,30-35
405,2020.12.07-045630,2020.11.29-231149,57780.0,68.802096,45.418737,7.053223,0.549498,0.000115,0.990626,AGP:31,AGP,AGP,30-35


**Load and Clean target prune datasets for current analyses and investigations.**

In [6]:
root_dir = '/home/franec94/Documents/thesys-siren/codebase/notebooks/analyze_quant_data/uniform_csv_files/tmp_quantized_datasets'
!ls {root_dir}

high-freq-siren_qatrlq_uniform_2020.12.14-174343.csv


In [7]:
# _ = interact(pick_tmp_dataset, datasets_list=datasets_list, a_dataset=dataset_path_wg)
dataset_path_wg = widgets.Text()
pick_tmp_dataset_wrapper(dataset_path_wg)

interactive(children=(Dropdown(description='datasets_list', options=('high-freq-siren_qatrlq_uniform_2020.12.1…

In [8]:
cmprss_df = get_selected_dataset(dataset_path_wg.value)
selected_columns = "date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class".split(",")
cmprss_df[selected_columns].head(5)

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class
0,2021.01.12-222903,2020.12.14-174343,16420.0,19.552274,29.491423,2.004395,3.991230,0.004508,0.808308,AGP+QATRLQ:S:PCW
1,2021.01.12-222940,2020.12.14-174343,16489.0,19.634437,39.044617,2.012817,3.974528,0.000499,0.970982,AGP+QATRLQ:S:PCW
2,2021.01.12-223018,2020.12.14-174343,16352.0,19.471303,29.346378,1.996094,4.007828,0.004665,0.793262,AGP+QATRLQ:S:NNPCW
3,2021.01.12-223053,2020.12.14-174343,16486.0,19.630864,37.734865,2.012451,3.975252,0.000674,0.961794,AGP+QATRLQ:S:NNPCW
4,2021.01.12-223129,2020.12.14-174343,16404.0,19.533222,26.188218,2.002441,3.995123,0.009621,0.731177,AGP+QATRLQ:S:PCW


In [9]:
cmprss_df.columns

Index(['date', 'init-from', 'size(byte)', 'footprint(%)', 'psnr', 'bpp', 'CR',
       'mse', 'ssim', 'cmprss-class', 'cmprss-class-2', 'prune_techs',
       'prune_rate_intervals', 'cmprss-class-3'],
      dtype='object')

In [10]:
selected_columns = "date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2,cmprss-class-3,prune_rate_intervals".split(",")
cmprss_df = cmprss_df[selected_columns] # cmprss_df[selected_columns].head(5)

In [11]:
show_hist_or_pie_quant_tech(cmprss_df)

interactive(children=(Checkbox(value=True, description='hist_or_pie_chart'), Dropdown(description='attribute',…

In [12]:
jpeg_df_tmp = pd.concat([jpeg_df, siren_bsln_df], axis = 0)
optional_dfs_dict = dict(
    jpeg_df = jpeg_df,
    siren_df = siren_bsln_df,
)
show_scatter_plot(cmprss_df, jpeg_df=jpeg_df_tmp, optional_dfs_dict=optional_dfs_dict)

interactive(children=(Checkbox(value=False, description='flag_add_jpeg_data'), Checkbox(value=False, descripti…

In [18]:
jpeg_df_tmp = pd.concat([jpeg_df, siren_bsln_df], axis = 0)
optional_dfs_dict = dict(
    jpeg_df = jpeg_df,
    siren_df = siren_bsln_df,
)
show_box_plot(cmprss_df, jpeg_df = jpeg_df, optional_dfs_dict=optional_dfs_dict)

interactive(children=(Checkbox(value=False, description='flag_add_jpeg_data'), Checkbox(value=False, descripti…

### Comparing table
---

In [14]:
baseline_data = ['2020.11.29-231149','-',20995*32/8,100.,49.16,20995*32/(256*256), (256*256*8)/(20995*32), 4.86e-5,9.96e-1,'Baseline-n_hf64-n_hl5', 'BASELINE']
columns = 'date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2'.split(",")
baseline_data_df = pd.DataFrame([baseline_data], columns = columns)
baseline_data_df

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2
0,2020.11.29-231149,-,83980.0,100.0,49.16,10.251465,0.780376,0.000049,0.996,Baseline-n_hf64-n_hl5,BASELINE


In [17]:
create_resulting_table_for_comparison_reasons_2(baseline_data_df, cmprss_df, jpeg_df, agp_df = None)

interactive(children=(Dropdown(description='target_class', options=('cmprss-class', 'cmprss-class-2', 'prune_r…

## References